# 模块、层和模型简介

要进行 TensorFlow 机器学习，可能需要定义、保存和恢复模型。<br>
模型是：
1. 一个在张量上进行某些计算的函数（前向传递）
2. 一些可以更新以响应训练的变量


In [4]:
import tensorflow as tf
from datetime import datetime

## 在 TensorFlow 中定义模型和层

大多数模型都由层组成。<br>
层是具有已知数学结构的函数，可以重复使用并且具有可训练的变量。
<br>在 TensorFlow 中，层和模型的大多数高级实现（例如 Keras 或 Sonnet）都在以下同一个基础类上构建：tf.Module。

---

In [2]:
# 定义自己的模型
class SimpleMoudle(tf.Module):
    def __init__(self,name=None):
        super().__init__(name=name)
        self.a_variable=tf.Variable(5.0,name="train_me")
        self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_variable
simpleMoudle=SimpleMoudle(name="simple")
simpleMoudle(tf.constant(5.0))


<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

```__call__``` 并无特殊之处，只是其行为与 Python 可调用对象类似；您可以使用任何函数来调用模型。<br><br>
您可以出于任何原因开启和关闭变量的可训练性，包括在微调过程中冻结层和变量。<br><br>


In [45]:
class Dense(tf.Module):
    def __init__(self,in_features, out_features,name=None):
        super().__init__(name=name)
        self.w=tf.Variable(
            tf.random.normal([in_features,out_features]),name='w'
        )
        self.b=tf.Variable(tf.zeros([out_features]),name='b'
        )
    def __call__(self,x):
        y=tf.matmul(x,self.w)+self.b
        return tf.nn.relu(y)

In [46]:
class SequentialMoudle(tf.Module):
    def __init__(self,name=None):
        super().__init__(name=name)
        self.dense_1=Dense(in_features=3,out_features=3)
        self.dense_2=Dense(in_features=3,out_features=2)
    def __call__(self,x):
        x=self.dense_1(x)
        return self.dense_2(x)


In [47]:
my_model=SequentialModule(name="the_modle")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[4.9118958 5.9864974]], shape=(1, 2), dtype=float32)


## 等待创建变量
通过将变量创建推迟到第一次使用特定输入形状调用模块时，您将无需预先指定输入大小。

In [52]:
class FlexibleDenseModule(tf.Module):
    def __init__(self,out_features,name=None):
        super().__init__(name=name)
        self.build=False
        self.out_features=out_features
    def __call__(self,x):
        if not self.build:
            self.w=tf.Variable(
                tf.random.normal([x.shape[-1], self.out_features]), name='w'
            )
            self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
            self.is_built = True
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)



In [53]:
# Used in a module
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = FlexibleDenseModule(out_features=3)
    self.dense_2 = FlexibleDenseModule(out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))


Model results: tf.Tensor([[0.       2.568204]], shape=(1, 2), dtype=float32)


## 保存权重

您可以将 tf.Module 保存为检查点和 SavedModel。<br><br>

检查点即是权重（即模块及其子模块内部的变量集的值）。

In [56]:
name='checkpoint'
checkpoint=tf.train.Checkpoint(model=my_model)
checkpoint.write(name)

'checkpoint'

## 创建 SavedModel

共享经过完全训练的模型的推荐方式是使用 SavedModel。<br><br>
SavedModel 包含函数集合与权重集合。您可以保存刚刚创建的模型。<br><br>
因此，利用 SavedModel，您可以使用 tf.Module 保存 TensorFlow 权重和计算图，随后再次加载它们。

In [ ]:
# 保存
tf.saved_model.save(my_model, "the_saved_model")
# 加载
new_model = tf.saved_model.load("the_saved_model")


## Keras 模型和层

可以在 tf.Module 上构建自己的高级 API，而我们已经拥有这些 API。<br><br>


### Keras 层
tf.keras.layers.Layer 是所有 Keras 层的基类，它继承自 tf.Module。<br><br>

您只需换出父项，然后将 __call__ 更改为 call 即可将模块转换为 Keras 层：<br><br>
```
class MyDense(tf.keras.layers.Layer):
  # Adding **kwargs to support base Keras layer arguemnts
  def __init__(self, in_features, out_features, **kwargs):
    super().__init__(**kwargs)

    # This will soon move to the build step; see below
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

simple_layer = MyDense(name="simple", in_features=3, out_features=3)

```

### build 步骤
在您确定输入形状之前，等待创建变量在许多情况下十分方便。<br><br>

Keras 层具有额外的生命周期步骤，可让您在定义层时获得更高的灵活性。这是在 build() 函数中定义的。<br><br>

build 仅被调用一次，而且是使用输入的形状调用的。它通常用于创建变量（权重）。<br><br>

您可以根据输入的大小灵活地重写上面的 MyDense 层。<br><br>
```
class FlexibleDense(tf.keras.layers.Layer):
  # Note the added `**kwargs`, as Keras supports many arguments
  def __init__(self, out_features, **kwargs):
    super().__init__(**kwargs)
    self.out_features = out_features

  def build(self, input_shape):  # Create the state of the layer (weights)
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

  def call(self, inputs):  # Defines the computation from inputs to outputs
    return tf.matmul(inputs, self.w) + self.b

# Create the instance of the layer
flexible_dense = FlexibleDense(out_features=3)

```